# Composite Model Evaluation

This notebook dynamically consolidates all model evaluation results from the `/evals` folder into a single Excel file with Winner and Runner-up columns.

**Features:**
- Uses `sample_questions.xlsx` as the master source for Query and GroundTruth columns
- Maintains original query order from `sample_questions.xlsx` (133 queries)
- Automatically discovers all `*_eval.xlsx` files in `/evals` folder
- Dynamically detects `*_Response` and `*_scores` columns
- Maps evaluation data by 'Query' column for proper alignment
- Determines Winner and Runner-up using winner.py logic
- Includes all queries from master file, even if missing from evaluation files
- Exports consolidated results to Excel in original query order

In [29]:
import pandas as pd
import os
import glob
from pathlib import Path
import ast
import numpy as np
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# Import winner logic
import winner

In [30]:
def discover_eval_files(evals_dir: str = "evals") -> List[str]:
    """
    Dynamically discover all evaluation Excel files in the evals directory.
    
    Args:
        evals_dir: Directory containing evaluation files
        
    Returns:
        List of file paths matching *_eval.xlsx pattern
    """
    pattern = os.path.join(evals_dir, "*_eval.xlsx")
    files = glob.glob(pattern)
    
    print(f"📁 Found {len(files)} evaluation files:")
    for file in files:
        print(f"  - {os.path.basename(file)}")
    
    return files

In [31]:
def extract_model_name_from_filename(filepath: str) -> str:
    """
    Extract model identifier from filename.
    
    Args:
        filepath: Path to evaluation file
        
    Returns:
        Model identifier (e.g., 'cohere_embed_v_4' from 'cohere_embed_v_4_eval.xlsx')
    """
    basename = os.path.basename(filepath)
    # Remove _eval.xlsx suffix
    model_name = basename.replace("_eval.xlsx", "")
    return model_name

In [32]:
def detect_model_columns(df: pd.DataFrame) -> Tuple[List[str], List[str], List[str]]:
    """
    Dynamically detect model response and scores columns.
    
    Args:
        df: DataFrame to analyze
        
    Returns:
        Tuple of (response_columns, scores_columns)
    """
    response_columns = [col for col in df.columns if col.endswith('_Response')]
    scores_columns = [col for col in df.columns if col.endswith('_scores')]
    context_columns = [col for col in df.columns if col.endswith('_Context')]
    
    return response_columns, scores_columns, context_columns

In [33]:
def extract_model_names_from_columns(columns: List[str], suffix: str) -> List[str]:
    """
    Extract model names from column names by removing suffix.
    
    Args:
        columns: List of column names
        suffix: Suffix to remove (e.g., '_Response', '_scores')
        
    Returns:
        List of model names
    """
    return [col.replace(suffix, '') for col in columns]

In [ ]:
def decide_winner_and_runner_up(model_scores: Dict[str, Dict[str, float]]) -> Tuple[str, str, Dict[str, float]]:
    """
    Modified winner logic to return separate Winner and Runner-up.
    
    Args:
        model_scores: Dictionary of model scores
        
    Returns:
        Tuple of (winner, runner_up, aggregate_scores)
    """
    if len(model_scores) == 0:
        return "No_Data", "No_Data", {}
    
    if len(model_scores) == 1:
        single_winner = list(model_scores.keys())[0]
        return single_winner, "No_Runner_Up", {}
    
    try:
        # Use existing winner logic to get aggregate scores
        _, aggregate_scores = winner.decide_winner(model_scores)
        
        # Sort by aggregate score to get winner and runner-up
        sorted_models = sorted(aggregate_scores.items(), key=lambda x: x[1], reverse=True)
        
        top_winner = sorted_models[0][0]
        runner_up = sorted_models[1][0] if len(sorted_models) > 1 else "No_Runner_Up"

        # Sort aggregate_scores dictionary in descending order by score
        aggregate_scores = dict(sorted(aggregate_scores.items(), key=lambda x: x[1], reverse=True))
        aggregate_scores = {k: round(v,2) for k, v in aggregate_scores.items()}

        return top_winner, runner_up, aggregate_scores
        
    except Exception as e:
        print(f"❌ Error in winner selection: {e}")
        return "Error", "Error", {}

In [35]:
def parse_scores_string(scores_str: str) -> Dict[str, float]:
    """
    Parse scores string to dictionary.
    
    Args:
        scores_str: String representation of scores dictionary
        
    Returns:
        Dictionary of scores
    """
    if pd.isna(scores_str) or not scores_str.strip():
        return {}
    
    try:
        # If it's already a dictionary, return as is
        if isinstance(scores_str, dict):
            return scores_str
        
        # Parse string representation
        if isinstance(scores_str, str):
            scores_str = scores_str.strip()
            if scores_str.startswith('{') and scores_str.endswith('}'):
                return ast.literal_eval(scores_str)
        
        return {}
    except Exception as e:
        print(f"⚠️ Error parsing scores: {e}")
        return {}

In [36]:
def load_and_process_eval_file(filepath: str) -> pd.DataFrame:
    """
    Load and process a single evaluation file.
    
    Args:
        filepath: Path to evaluation Excel file
        
    Returns:
        Processed DataFrame
    """
    try:
        df = pd.read_excel(filepath)
        print(f"📊 Loaded {len(df)} rows from {os.path.basename(filepath)}")
        
        # Detect model columns
        response_cols, scores_cols, context_cols = detect_model_columns(df)
        print(f"  Response columns: {response_cols}")
        print(f"  Scores columns: {scores_cols}")
        print(f"  Context columns: {context_cols}")
        
        return df
        
    except Exception as e:
        print(f"❌ Error loading {filepath}: {e}")
        return pd.DataFrame()

In [37]:
def consolidate_evaluations(evals_dir: str = "evals", questions_file: str = "sample_questions.xlsx") -> pd.DataFrame:
    """
    Main function to consolidate all evaluation files using sample_questions.xlsx as master.
    
    Args:
        evals_dir: Directory containing evaluation files
        questions_file: Path to master questions file with Query and Ground_Truth columns
        
    Returns:
        Consolidated DataFrame with queries in original order
    """
    # Load master questions file to get queries and ground truth in original order
    try:
        master_df = pd.read_excel(questions_file)
        print(f"📋 Loaded {len(master_df)} queries from master file: {questions_file}")
        print(f"Columns: {list(master_df.columns)}")
        
        if not any(word in col.lower() for col in master_df.columns for word in ['query','queries','question','questions']):
            print("❌ 'Query' column not found in master file")
            return pd.DataFrame()
            
        # Get queries in original order
        master_queries = master_df['Query'].dropna().tolist()
        ground_truths = master_df.get('Ground_Truth', pd.Series(['']*len(master_queries))).fillna('').tolist()
        
    except Exception as e:
        print(f"❌ Error loading master questions file: {e}")
        return pd.DataFrame()
    
    # Discover evaluation files
    eval_files = discover_eval_files(evals_dir)
    
    if not eval_files:
        print("❌ No evaluation files found")
        return pd.DataFrame()
    
    # Load all eval files
    all_dfs = []
    for filepath in eval_files:
        df = load_and_process_eval_file(filepath)
        if not df.empty:
            # Add source file info
            df['_source_file'] = os.path.basename(filepath)
            all_dfs.append(df)
    
    if not all_dfs:
        print("❌ No valid evaluation files loaded")
        return pd.DataFrame()
    
    print(f"\n🔍 Processing {len(master_queries)} queries in original order from master file")
    
    # Consolidate data by query in original order
    consolidated_results = []
    
    for i, (query, ground_truth) in enumerate(zip(master_queries, ground_truths)):
        if pd.isna(query) or not str(query).strip():
            continue
            
        query_str = str(query).strip()
        
        # Initialize result row with master data
        result_row = {
            'Query': query_str,
            'GroundTruth': str(ground_truth) if pd.notna(ground_truth) else '',
            'RAG': 'Contextual RAG',
            'Context': ''
        }
        
        # Collect model data for this query from eval files
        model_scores = {}
        found_in_evals = False
        
        for df in all_dfs:
            # Find matching query in this dataframe
            matching_rows = df[df['Query'] == query_str]
            
            if matching_rows.empty:
                continue
                
            found_in_evals = True
            row = matching_rows.iloc[0]
            
            # Update context (use first non-empty value from eval files)
            if not result_row['Context'] and 'Context' in row and pd.notna(row['Context']):
                result_row['Context'] = row['Context']
            
            # Extract model responses and scores
            response_cols, scores_cols, context_cols = detect_model_columns(df)
            
            for response_col in response_cols:
                if response_col in row and pd.notna(row[response_col]):
                    result_row[response_col] = row[response_col]

            for context_col in context_cols:
                if context_col in row and pd.notna(row[context_col]):
                    result_row[context_col] = row[context_col]
            
            for scores_col in scores_cols:
                if scores_col in row and pd.notna(row[scores_col]):
                    result_row[scores_col] = row[scores_col]
                    
                    # Parse scores for winner calculation
                    model_name = scores_col.replace('_scores', '')
                    parsed_scores = parse_scores_string(row[scores_col])
                    
                    if parsed_scores:
                        model_scores[model_name] = parsed_scores
        
        # Determine winner and runner-up (only if we have evaluation data)
        if model_scores:
            winner, runner_up, aggregate_scores = decide_winner_and_runner_up(model_scores)
            result_row['Winner'] = winner
            result_row['Runner_Up'] = runner_up
            result_row['Scores'] = aggregate_scores
        else:
            # Query exists in master but not in evaluation files
            result_row['Winner'] = 'No_Evaluation_Data'
            result_row['Runner_Up'] = 'No_Evaluation_Data'
            result_row['Scores'] = {}
        
        consolidated_results.append(result_row)
        
        # Progress indicator for every 20 queries
        if (i + 1) % 20 == 0:
            print(f"  Processed {i + 1}/{len(master_queries)} queries...")
    
    # Convert to DataFrame
    consolidated_df = pd.DataFrame(consolidated_results)
    
    # Count queries with evaluation data
    evaluated_count = sum(1 for row in consolidated_results if row['Winner'] != 'No_Evaluation_Data')
    
    print(f"\n✅ Consolidated {len(consolidated_df)} queries in original order")
    print(f"📊 {evaluated_count} queries have evaluation data, {len(consolidated_df) - evaluated_count} missing from eval files")
    
    return consolidated_df

In [38]:
def save_consolidated_results(df: pd.DataFrame, output_file: str = "composite_model_evaluation_results.xlsx"):
    """
    Save consolidated results to Excel file.
    
    Args:
        df: Consolidated DataFrame
        output_file: Output file path
    """
    if df.empty:
        print("❌ No data to save")
        return
    
    try:
        df.to_excel(output_file, index=False)
        print(f"💾 Consolidated results saved to: {output_file}")
        print(f"📊 Total rows: {len(df)}")
        
        # Show column summary
        print(f"📋 Columns: {len(df.columns)}")
        print(f"  Core columns: Query, GroundTruth, RAG, Context, Winner, Runner_Up, Scores")
        
        # Show dynamic columns
        response_cols = [col for col in df.columns if col.endswith('_Response')]
        scores_cols = [col for col in df.columns if col.endswith('_scores')]
        context_cols = [col for col in df.columns if col.endswith('_Context')]
        
        print(f"  Response columns: {response_cols}")
        print(f"  Scores columns: {scores_cols}")
        print(f"  Scores columns: {context_cols}")
        
    except Exception as e:
        print(f"❌ Error saving results: {e}")

In [39]:
def show_winner_summary(df: pd.DataFrame):
    """
    Display winner and runner-up summary statistics.
    
    Args:
        df: Consolidated DataFrame
    """
    if df.empty or 'Winner' not in df.columns:
        print("❌ No winner data to summarize")
        return
    
    print("\n🏆 Winner Distribution:")
    winner_counts = df['Winner'].value_counts()
    for model, count in winner_counts.items():
        print(f"  {model}: {count} wins")
    
    if 'Runner_Up' in df.columns:
        print("\n🥈 Runner-Up Distribution:")
        runner_up_counts = df['Runner_Up'].value_counts()
        for model, count in runner_up_counts.items():
            print(f"  {model}: {count} runner-ups")

## Execute Consolidation

Run the consolidation process:

In [40]:
# Run the consolidation
print("🚀 Starting composite model evaluation consolidation...")
consolidated_df = consolidate_evaluations()

# Save results
if not consolidated_df.empty:
    save_consolidated_results(consolidated_df)
    
    # Show summary
    show_winner_summary(consolidated_df)
    
    # Display sample results
    print("\n📋 Sample Results:")
    display_cols = ['Query', 'Winner', 'Runner_Up']
    available_cols = [col for col in display_cols if col in consolidated_df.columns]
    print(consolidated_df[available_cols].head(10))
    
else:
    print("❌ No consolidated results generated")

🚀 Starting composite model evaluation consolidation...
📋 Loaded 133 queries from master file: sample_questions.xlsx
Columns: ['Query', 'Ground_Truth']
📁 Found 3 evaluation files:
  - gpt41_openai_embed_eval.xlsx
  - deepseek_openai_embed_eval.xlsx
  - cohere_embed_v_4_eval.xlsx
📊 Loaded 133 rows from gpt41_openai_embed_eval.xlsx
  Response columns: ['gpt-4.1_Response']
  Scores columns: ['gpt-4.1_scores']
  Context columns: ['gpt-4.1_Context']
📊 Loaded 133 rows from deepseek_openai_embed_eval.xlsx
  Response columns: ['deepseek_Response']
  Scores columns: ['deepseek_scores']
  Context columns: ['deepseek_Context']
📊 Loaded 133 rows from cohere_embed_v_4_eval.xlsx
  Response columns: ['cohere_Response']
  Scores columns: ['cohere_scores']
  Context columns: ['cohere_Context']

🔍 Processing 133 queries in original order from master file
  Processed 20/133 queries...
  Processed 40/133 queries...
  Processed 60/133 queries...
  Processed 80/133 queries...
  Processed 100/133 queries...
 